In [ ]:
import gym
from gym import spaces
from gym.spaces import Box
import pandas as pd
import numpy as np
import tensorflow as tf
import os
n_actions = 7
n_states = 6
A = [[0.992,0.0018,0,0,0,0],
              [0.0023,0.9919,0.0043,0,0,0],
              [0,-0.0042,1.0009,0.0024,0,0],
              [0,0,0.0013,0.9979,0,0],
              [0,0,0,0,0.9972,0],
              [0,0,0,0,0,0.9953]]

B =[[1.0033,0,0,0,0,0,-0.2175],
              [0,1.0460,0,0,0,0,-0.0788],
              [0,0,1.0326,0,0,0,-0.0020],
              [0,0,0,0.4798,0,0,-0.0669],
              [0,0,0,0,0.8882,0,0.1273],
              [0,0,0,0,0,1.1699,-0.1792]]
Q = np.identity(n_states)
R =  np.identity(n_actions)
class SmallSS(gym.Env):
    def __init__(self):
        self.action_space = Box(low = 0 , high = 1, shape=(n_actions,))
        self.observation_space = Box(low = 50 , high = 300, shape=(n_states,))
        self.state = np.repeat(60,n_states)
        self.simulation_length = 1000
        self.A = A
        self.B = B
        self.Q = Q
        self.R = R
        self.reference = np.array([150,160,165,170,175,180])
    def step(self, action):
        self.state = A@self.state + B@action 
        self.simulation_length -= 1
        curr_diff = abs(self.state - self.reference)
        upplim = np.repeat(220,n_states)
        lowlim = np.repeat(110,n_states)
        up_constr = np.mean(self.state-upplim)
        low_constr = np.mean(lowlim-self.state)
        reward = (curr_diff@Q@curr_diff) + (action@R@action) + 0.1*up_constr + 0.1*low_constr
        
        if self.simulation_length <=0:
            done = True
        else:
            done = False

        self.state = tf.clip_by_value(self.state, 50, 300).numpy()
        info ={}

        return self.state, reward, done, info 
    
    def render(self):
        pass
    def reset(self):
        self.state = np.repeat(60,n_states)
        self.simulation_length = 1000
        return self.state
    
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam

import numpy as np

class ReplayBuffer:
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_cntr = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape))
        self.new_state_memory = np.zeros((self.mem_size, *input_shape))
        self.action_memory = np.zeros((self.mem_size, n_actions))
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=bool)
        self.state_running_avg = [0]
        self.state_running_std = [1]
        self.m1 = [0]
        self.m2 = [0]
        self.s1 = [1]
        self.s2 = [1]

    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size

        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done

        self.mem_cntr += 1

    def sample_buffer(self, batch_size):
        max_mem = min(self.mem_cntr, self.mem_size)

        batch = np.random.choice(max_mem, batch_size, replace=False)

        states = self.state_memory[batch]
        states_ = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        dones = self.terminal_memory[batch]
        
        state_mean, state_std = np.mean(states), np.std(states)
        states  = (states - state_mean)/state_std
            
        new_state_mean, new_state_std = np.mean(states_), np.std(states_)
        states_ = (states_ - new_state_mean)/new_state_std
        
        self.m1 = [state_mean]
        self.m2 = [new_state_mean]

        self.s1 = [state_std]
        self.s2 = [new_state_std]

        self.state_running_avg = np.mean([self.state_running_avg,self.m1,self.m2],axis=0,dtype=object)
        self.state_running_std = np.mean([self.state_running_std,self.s1,self.s2],axis=0,dtype=object)
        

        return states, actions, rewards, states_, dones
    
    def getstuff(self):
            return self.state_running_avg , self.state_running_std
    
class Agent:
    def __init__(self, input_dims, alpha=0.002, beta=0.0005, env=None,
                 gamma=0.99, n_actions=7, max_size=1000000, tau=0.005,
                 fc1=1000, fc2=800, batch_size=128, noise=0.3):
        self.gamma = gamma
        self.tau = tau
        self.memory = ReplayBuffer(max_size, input_dims, n_actions)
        self.batch_size = batch_size
        self.n_actions = n_actions
        self.noise = noise
        self.max_action = env.action_space.high[0]
        self.min_action = env.action_space.low[0]

        self.actor = ActorNetwork(n_actions=n_actions, name='actor')
        self.critic = CriticNetwork(name='critic')
        self.target_actor = ActorNetwork(n_actions=n_actions, name='target_actor')
        self.target_critic = CriticNetwork(name='target_critic')

        self.actor.compile(optimizer=Adam(learning_rate=alpha))
        self.critic.compile(optimizer=Adam(learning_rate=beta))
        self.target_actor.compile(optimizer=Adam(learning_rate=alpha))
        self.target_critic.compile(optimizer=Adam(learning_rate=beta))

        self.actor_loss = []
        self.critic_loss = []

        self.update_network_parameters(tau=1)

    def update_network_parameters(self, tau=None):
        if tau is None:
            tau = self.tau

        weights = []
        targets = self.target_actor.weights
        for i, weight in enumerate(self.actor.weights):
            weights.append(weight * tau + targets[i]*(1-tau))
        self.target_actor.set_weights(weights)

        weights = []
        targets = self.target_critic.weights
        for i, weight in enumerate(self.critic.weights):
            weights.append(weight * tau + targets[i]*(1-tau))
        self.target_critic.set_weights(weights)

    def remember(self, state, action, reward, new_state, done):
        self.memory.store_transition(state, action, reward, new_state, done)

    def save_models(self):
        print('... saving models ...')
        self.actor.save_weights(self.actor.checkpoint_file)
        self.target_actor.save_weights(self.target_actor.checkpoint_file)
        self.critic.save_weights(self.critic.checkpoint_file)
        self.target_critic.save_weights(self.target_critic.checkpoint_file)

    def load_models(self):
        print('... loading models ...')
        self.actor.load_weights(self.actor.checkpoint_file)
        self.target_actor.load_weights(self.target_actor.checkpoint_file)
        self.critic.load_weights(self.critic.checkpoint_file)
        self.target_critic.load_weights(self.target_critic.checkpoint_file)

    def choose_action(self, observation, evaluate=False):
        state = tf.convert_to_tensor([observation], dtype=tf.float32)
        actions = self.actor(state)
        if not evaluate:
            actions += tf.random.normal(shape=[self.n_actions],
                                        mean=0.0, stddev=self.noise)
        # note that if the env has an action > 1, we have to multiply by
        # max action at some point
        actions = tf.clip_by_value(actions, self.min_action, self.max_action)

        return actions[0]
    
    def reduce_exp_noise(self):
        self.noise = self.noise*0.9

    def learn(self):
        if self.memory.mem_cntr < self.batch_size:
            return

        state, action, reward, new_state, done = \
            self.memory.sample_buffer(self.batch_size)

        states = tf.convert_to_tensor(state, dtype=tf.float32)
        states_ = tf.convert_to_tensor(new_state, dtype=tf.float32)
        rewards = tf.convert_to_tensor(reward, dtype=tf.float32)
        actions = tf.convert_to_tensor(action, dtype=tf.float32)

        with tf.GradientTape() as tape:
            target_actions = self.target_actor(states_)
            critic_value_ = tf.squeeze(self.target_critic(
                                states_, target_actions), 1)
            critic_value = tf.squeeze(self.critic(states, actions), 1)
            target = rewards + self.gamma*critic_value_*(1-done)
            critic_loss = keras.losses.MSE(target, critic_value)

        critic_network_gradient = tape.gradient(critic_loss,
                                                self.critic.trainable_variables)
        self.critic.optimizer.apply_gradients(zip(
            critic_network_gradient, self.critic.trainable_variables))

        with tf.GradientTape() as tape:
            new_policy_actions = self.actor(states)
            actor_loss = self.critic(states, new_policy_actions)
            actor_loss = tf.math.reduce_mean(actor_loss)

        actor_network_gradient = tape.gradient(actor_loss,
                                               self.actor.trainable_variables)
        self.actor.optimizer.apply_gradients(zip(
            actor_network_gradient, self.actor.trainable_variables))

        self.update_network_parameters()

        self.actor_loss.append(actor_loss)
        self.critic_loss.append(critic_loss)

        import os
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense

class CriticNetwork(keras.Model):
    def __init__(self, fc1_dims=1000, fc2_dims=800,
            name='critic', chkpt_dir='/Users/elenizavrakli/Documents/GitHub/DDPG'):
        super(CriticNetwork, self).__init__()
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims

        self.model_name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, 
                    self.model_name+'_ddpg')

        self.fc1 = Dense(self.fc1_dims, activation='relu')
        self.fc2 = Dense(self.fc2_dims, activation='relu')
        self.q = Dense(1, activation=None)

    def call(self, state, action):
        action_value = self.fc1(tf.concat([state, action], axis=1))
        action_value = self.fc2(action_value)

        q = self.q(action_value)

        return q

class ActorNetwork(keras.Model):
    def __init__(self, fc1_dims=1000, fc2_dims=800, n_actions=7, name='actor',
            chkpt_dir='/Users/elenizavrakli/Documents/GitHub/DDPG'):
        super(ActorNetwork, self).__init__()
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions

        self.model_name = name
        self.checkpoint_dir = chkpt_dir
        self.checkpoint_file = os.path.join(self.checkpoint_dir, 
                    self.model_name+'_ddpg')

        self.fc1 = Dense(self.fc1_dims, activation='relu')
        self.fc2 = Dense(self.fc2_dims, activation='relu')
        self.mu = Dense(self.n_actions, activation='sigmoid',kernel_initializer =  tf.random_uniform_initializer(minval=-0.003, maxval=0.003))

    def call(self, state):
        prob = self.fc1(state)
        prob = self.fc2(prob)

        mu = self.mu(prob)

        return mu

import numpy as np
import matplotlib.pyplot as plt

def plot_learning_curve(x, scores, figure_file):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')
    #plt.savefig(figure_file)


if __name__ == '__main__':
    env = SmallSS()
    agent = Agent(input_dims=env.observation_space.shape, env=env,
            n_actions=env.action_space.shape[0])
    n_games = 600

    figure_file = 'plots/ddpg_temp.png'

    best_score = env.reward_range[0]
    score_history = []
    load_checkpoint = False

    if load_checkpoint:
        n_steps = 0
        while n_steps <= agent.batch_size:
            observation = env.reset()
            action = env.action_space.sample()
            observation_, reward, done, info = env.step(action)
            agent.remember(observation, action, reward, observation_, done)
            n_steps += 1
        agent.learn()
        agent.load_models()
        evaluate = True
    else:
        evaluate = False

    for i in range(n_games):
        max_state = 0
        observation = env.reset()
        done = False
        score = 0
        while not done:
            if i<=29:
                action = env.action_space.sample()
            else:
                m,s = agent.memory.getstuff()
                observation_n = (observation-m)/s
                action = agent.choose_action(observation_n, evaluate).numpy()
            if np.mean(observation)>max_state:
                max_state = np.mean(observation)
            observation_, reward, done, info = env.step(action)
            score += reward
            agent.remember(observation, action, reward, observation_, done)
            if not load_checkpoint:
                agent.learn()
            observation = observation_
        if i%100 == 0:
            agent.reduce_exp_noise()
        

        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score:
            best_score = avg_score
            #if not load_checkpoint:
             #   agent.save_models()

        print('episode ', i, 'score %.1f' % score, 'avg score %.1f' % avg_score)
        print('Max mean state of episode',max_state)

    if not load_checkpoint:
        x = [i+1 for i in range(n_games)]
        plot_learning_curve(x, score_history, figure_file)

In [ ]:
state_history=[]
evaluate = True
observation = env.reset()
done = False
score_history = []
i=1
gamma = 0.99
while not done:
    m,s = agent.memory.getstuff()
    observation_n = (observation-m)/s
    action = agent.choose_action(observation_n, evaluate).numpy()
    observation_, reward, done, info = env.step(action)
    score += pow(gamma,i)*reward
    score_history.append(score)
    observation = observation_
    state_history.append(observation)
    i += 1
import matplotlib.pyplot as plt

plt.plot([i+1 for i in range(1000)], np.array(state_history)[0:1000,:])
plt.ylabel('Heater temperatures')
plt.xlabel('time steps')
#plt.savefig("trajectories.pdf")

x=np.array(state_history)[999,:]
ref=np.array([150,160,165,170,175,180])
print(x)
print(x-ref)
print(score)

In [ ]:
plt.plot([i+1 for i in range(1000)], np.array(score_history)[0:1000])
plt.ylabel('Sum of discounted costs')
plt.xlabel('time steps')
#plt.savefig("cost.pdf")

In [ ]:
plt.plot([i+1 for i in range(np.shape(agent.critic_loss)[0])], np.array(agent.critic_loss))
plt.title("Critic loss")
plt.ylabel('Loss')
plt.xlabel('Training Iteration')
#plt.savefig('critic.pdf')

In [ ]:
plt.plot([i+1 for i in range(np.shape(agent.actor_loss)[0])], np.array(agent.actor_loss))
plt.title("Actor loss")
plt.ylabel('Loss')
plt.xlabel('Training Iteration')
#plt.savefig('actor.pdf')